In [61]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
import sys
import numpy as np

In [ ]:
DIR = "/Users/damoncrockett/pga-golfers/"

In [54]:
letters = [item for item in string.ascii_uppercase]

In [55]:
urlstring = "http://newsok.sportsdirectinc.com/golf/pga-players.aspx?page=/data/pga/players/"
urls = [urlstring + item + "_players.html" for item in letters]

In [56]:
counter=0
cellstrings = []
for url in urls:
    counter+=1
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    cells = soup.find_all('td', {'class' : 'sdi-datacell'})
    cells = cells[1:]
    for cell in cells:
        cell = cell.get_text()
        cell = cell.replace("\n\r","")
        cell = "".join(cell.split())
        cellstrings.append(cell)

In [96]:
df = pd.DataFrame({
        "name":[cellstrings[i] for i in np.arange(0,7310,5)],
        "height":[cellstrings[i] for i in np.arange(1,7310,5)],
        "weight":[cellstrings[i] for i in np.arange(2,7310,5)],
        "dob":[cellstrings[i] for i in np.arange(3,7310,5)],
        "hometown":[cellstrings[i] for i in np.arange(4,7310,5)]
    }
                  )

In [ ]:
#df.to_csv(DIR+"raw.csv",index=False)

In [97]:
from dateutil import parser

In [98]:
df['dob'] = df.dob.apply(parser.parse)

In [99]:
import datetime

In [100]:
df['diff'] = [datetime.datetime.today() - item for item in df.dob]

In [101]:
df.head()

,dob,height,hometown,name,weight,diff
0,1973-01-16,"6'2""","Providence,USA","Adamonis,Brad",215lbs,16225 days 01:29:58.510372
1,1975-08-27,"6'3""","Bartlesville,USA","Adams,Blake",205lbs,15272 days 01:29:58.510443
2,1979-05-27,"5'7""",USA,"AdamsIII,Frank",165lbs,13903 days 01:29:58.510490
3,1981-08-01,"5'7""","Chicago,USA","Affrunti,Joe",155lbs,13106 days 01:29:58.510522
4,2017-06-19,"6'2""","Roanoke,USA","Agee,Brian",168lbs,0 days 01:29:58.510553


In [102]:
df['days'] = [item / datetime.timedelta(days=1) for item in df['diff']]

In [150]:
df['years'] = [item/float(365) for item in df['days']]

In [155]:
def nullify(x):
    if x <= 1:
        return None
    else:
        return x

In [156]:
df['years'] = df.years.apply(nullify)

In [132]:
def country(x):
    if x=="":
        return None
    else:
        s = x.split(",")
        if len(s)==1:
            return x
        elif len(s)==2:
            return s[1]
        else:
            return None

In [133]:
df['country'] = df.hometown.apply(country)

In [161]:
def weight(x):
    if x=='':
        return None
    else:
        x = x.rstrip('lbs')
        x = int(x)
        return x

In [162]:
df['weight'] = df.weight.apply(weight)

In [166]:
def height(x):
    if x=='':
        return None
    else:
        x = x.rstrip('"')
        pair = x.split("'")
        feet = pair[0]
        inches = pair[1]
        total = int(feet)*12 + int(inches)
        return total

In [168]:
df['height'] = df.height.apply(height)

In [169]:
df.head()

,dob,height,hometown,name,weight,diff,days,years,country
0,1973-01-16,74.0,"Providence,USA","Adamonis,Brad",215.0,16225 days 01:29:58.510372,16225.062483,44.452226,USA
1,1975-08-27,75.0,"Bartlesville,USA","Adams,Blake",205.0,15272 days 01:29:58.510443,15272.062483,41.841267,USA
2,1979-05-27,67.0,USA,"AdamsIII,Frank",165.0,13903 days 01:29:58.510490,13903.062483,38.090582,USA
3,1981-08-01,67.0,"Chicago,USA","Affrunti,Joe",155.0,13106 days 01:29:58.510522,13106.062483,35.907021,USA
4,2017-06-19,74.0,"Roanoke,USA","Agee,Brian",168.0,0 days 01:29:58.510553,0.062483,NaN,USA


In [170]:
tmp = df[['height','weight','years','country']]

In [172]:
tmp.to_csv(DIR+"data.csv",index=False)